## Set to directory with python scripts

In [1]:
scripts_path = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts/'

# Path to save figures
path_to_save = '/'.join(scripts_path.split('/')[:-2]) + '/'

# Path to save supplementary and figure data folders
save_path = '/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/'
%cd $scripts_path

/Users/jendawk/Dropbox (MIT)/C Diff Recurrence Paper/Analyses/scripts


## Load modules

In [2]:
import scipy.stats as st
from collections import Counter
import sklearn
from matplotlib import cm
import scipy
import  itertools
from datetime import datetime
from seaborn import clustermap
from scipy.cluster.hierarchy import linkage
from sklearn.linear_model import LogisticRegression
import os
import time
import pickle as pkl
import pandas as pd
from dataLoader import *
from basic_data_methods_helper import *
from skbio.stats.distance import permanova, DistanceMatrix, anosim
from skbio.stats.distance import mantel
from statsmodels.stats.multitest import multipletests
from matplotlib.collections import LineCollection
from Bio import Phylo
import re
from statistics import mode
import seaborn as sns
from sksurv.linear_model import CoxnetSurvivalAnalysis, CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored
import matplotlib.lines as mlines
import seaborn as sns
from skbio.diversity.alpha import chao1

# Set font for figures
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Arial']})

## Load data

In [3]:
# set data_path to point to directory with data
data_path = save_path + "Data/"

# Option to change filtering criteria
dl = dataLoader(path = data_path, pt_perc = {'metabs': .25, '16s': .1, 'scfa': 0, 'toxin':0}, meas_thresh = 
                {'metabs': 0, '16s': 10, 'scfa': 0, 'toxin':0}, 
                var_perc = {'metabs': 50, '16s': 5, 'scfa': 0, 'toxin':0}, pt_tmpts = 1)

/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [4]:
# Number of features in each data set
for key in dl.week.keys():
    print(key + ': ' + str(dl.week[key][0]['x'].shape[1]))

metabs: 760
16s: 237
scfa: 8
toxin: 2
metabs_16s: 997
metabs_16s_scfa: 997
metabs_toxin: 762


# Table 1

### Table 1 - Demographics

In [10]:
targets = pd.Series(dl.targets_by_pt)
targets = targets.replace('Cleared','Non-recurrer')
targets = targets.replace('Recur', 'Recurrer')
demo_dict = {}
for feat in dl.demographics.columns.values:
    if len(np.unique(dl.demographics[feat]))<=4 or (dl.demographics[feat].dtypes!=int and dl.demographics[feat].dtypes!=float):
        for cat in np.unique(dl.demographics[feat]):
            N = np.sum((dl.demographics[feat]==cat)*(targets=='Non-recurrer'))
            c_re = np.sum((dl.demographics[feat]==cat)*(targets=='Recurrer'))
            demo_dict[(feat,cat)] = {'N_nonRec': N, 'N_Rec': c_re}
    else:
        demo_dict[(feat,'Mean')] = {'N': np.mean(dl.demographics[feat])}
        demo_dict[(feat,'STD')] = {'N': np.std(dl.demographics[feat])}
        demo_dict[(feat,'Median')] = {'N': np.median(dl.demographics[feat])}
        demo_dict[(feat,'Range')] = {'N': (np.min(dl.demographics[feat]),np.max(dl.demographics[feat]))}
        
df = pd.DataFrame(demo_dict).T
if not os.path.isdir(save_path + 'Main Tables'):
    os.mkdir(save_path + 'Main Tables')
df.to_csv(save_path + 'Main Tables/Table 1.csv')

/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expressions.py:205: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  f"evaluating in Python space because the {repr(op_str)} "
/Users/jendawk/miniconda3/envs/CodeBase/lib/python3.7/site-packages/pandas/core/computation/expre

### Table 1 - # Recurrers at each timepoint

In [39]:
rc_dict = {}
for week in dl.week['metabs'].keys():
    rc_dict[week]={}
    targets = dl.week['metabs'][week]['y']
    rc_dict[week]['Recurrers'] = np.sum(targets=='Recurrer')
    rc_dict[week]['Non-Recurrers'] = np.sum(targets=='Non-recurrer')
if not os.path.isdir(path_to_save + 'output_tables'):
    os.mkdir(path_to_save + 'output_tables')
pd.DataFrame(rc_dict).T.to_csv(save_path + 'Main Tables/Table 2.csv')

# Univariate analysis

## Univariate analysis for demographics and clinical variables

In [38]:
if not os.path.isdir(path_to_save + 'univariate_analysis'):
    os.mkdir(path_to_save + 'univariate_analysis')

# Univariate analysis for demographics
targets = dl.keys['metabs']['targets_by_pt']
x = dl.demographics
xx = dl.demographics[['Age','Sex','Race','BMI','Smoking status']]
df_out = univariate_w_chi2(xx, targets)
df_out.to_csv(path_to_save + 'univariate_analysis/demographics.csv')

# Univariate analysis for clinical variables
targets = dl.keys['metabs']['targets_by_pt']
x = dl.clinical
df_out = univariate_w_chi2(x, targets)
df_out.to_csv(path_to_save + 'univariate_analysis/clinical.csv')

## Export 16S data for DESeq2 in R

In [20]:
path_to_save_data=save_path + '/GenFigure&AnalysesData/'

if not os.path.isdir(path_to_save + 'scripts/inputs/'):
    os.mkdir(path_to_save + 'scripts/inputs/')
    
if not os.path.isdir(path_to_save + 'scripts/inputs/DEseq2'):
    os.mkdir(path_to_save + 'scripts/inputs/DEseq2')

weeks = [0,1,2]
plot = True

data_to_save = {}
key = '16s'
if key not in data_to_save.keys():
    data_to_save[key] = {}
for week in weeks:
#     if not isinstance(week, list):
    x = dl.week_filt[key][week]['x']
    y = (dl.week_filt[key][week]['y']=='Recurrer').astype('float')
#     else:
#         x,y,t = get_slope_data(dl.week_filt[key], week)
        
    if not os.path.isdir('inputs/DEseq2/'):
        os.mkdir('inputs/DEseq2/')
    x.to_csv(path_to_save +'scripts/inputs/DEseq2/counts_' + str(week).replace('.','-') + '.csv')
    y.to_csv(path_to_save +'scripts/inputs/DEseq2/col_' + str(week).replace('.','-') + '.csv')
    yy = pd.DataFrame(y, columns = ['Outcome'])
    both = pd.concat([x, yy], 1)
    if 'Week ' + str(week) not in data_to_save[key].keys():
        data_to_save[key]['Week ' + str(week)] = {}
    data_to_save[key]['Week ' + str(week)]['RvNR'] = both
    
# Export for DEseq2 in R
weeks = [(0,1),(1,2)]
key = '16s'
for outcome in ['Recurrer', 'Non-recurrer']:
    for week_pair in weeks:
        col0 = dl.week_filt[key][week_pair[0]]['x'].columns.values
        col1 = dl.week_filt[key][week_pair[1]]['x'].columns.values
        col_all = np.unique(np.concatenate([col0, col1]))
        ix0, ix1 = dl.week_filt[key][week_pair[0]]['x'].index.values, dl.week_filt[key][week_pair[1]]['x'].index.values
        
        x0, y0 = dl.keys[key]['data'][col_all].loc[ix0], dl.week_filt[key][week_pair[0]]['y']
        x1, y1 = dl.keys[key]['data'][col_all].loc[ix1], dl.week_filt[key][week_pair[1]]['y']
        x0 = x0.loc[y0.values == outcome]
        x1 = x1.loc[y1.values == outcome]
        x = pd.concat([x0, x1])
        y = [week_pair[0]]*x0.shape[0] + [week_pair[1]]*x1.shape[0]
        y = pd.Series((np.array(y)==week_pair[1]).astype('float'), index = x.index.values)
        if not os.path.isdir('inputs/DEseq2/'):
            os.mkdir('inputs/DEseq2/')
        x.to_csv(path_to_save +'scripts/inputs/DEseq2/counts_' + outcome +str(week_pair[0]) + '_' + str(week_pair[1]) + '.csv')
        y.to_csv(path_to_save +'scripts/inputs/DEseq2/col_' + outcome + str(week_pair[0]) + '_' + str(week_pair[1]) + '.csv')
        
        yy = pd.DataFrame(y, columns = ['Outcome'])
        both = pd.concat([x, yy], 1)
        if outcome not in data_to_save[key].keys():
            data_to_save[key][outcome] = {}
        data_to_save[key][outcome]['Week ' + str(week_pair[0]) + 'v' + str(week_pair[1])] = both
        
for key in data_to_save.keys():
    for key2 in data_to_save[key].keys():
        for key3 in data_to_save[key][key2].keys():
            val = pd.DataFrame(data_to_save[key][key2][key3])
            if os.path.isfile(path_to_save_data + '/' + 'univ_analysis_data.xlsx'):
                with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_data.xlsx', mode='a', engine='openpyxl') as writer:
                    val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
            else:
                with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_data.xlsx', mode = 'w', engine='openpyxl') as writer:
                    val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
        

## Reformat DEseq2 results

In [15]:
# Set path_deseq2 to path with R results
path_deseq2 = path_to_save + 'univariate_analysis/16s/deseq2/'

weeks = [0,1,2]
plot = False

key = '16s'
    
if not os.path.isdir(path_to_save + 'univariate_analysis/' + key):
    os.mkdir(path_to_save + 'univariate_analysis/' + key)
for file in os.listdir(path_deseq2):
    if file == '.DS_Store':
        continue
    week = file.split('res')[-1].split('.')[0]
    df_r = pd.read_csv(path_deseq2 + file, index_col = 0)
    df_sorted = df_r.sort_values(by = 'padj')
    taxa = return_taxa_names(df_sorted.OTU)
    df_sorted.insert(1, "Taxa", taxa, allow_duplicates=True)
    
    df_sorted.insert(2,"Direction", np.zeros(df_sorted.shape[0]).astype(str), True)
    
    if '_' in file:
        # Higher in cleared group (0) or first week
        df_sorted["Direction"].loc[df_sorted["log2fold"]<0] = 'Higher in week ' + file.split('_')[0].split('rer')[1]

        # Higher in recurred group (1) or second week
        df_sorted["Direction"].loc[df_sorted["log2fold"]>0] = 'Higher in week ' + file.split('_')[1].split('.')[0]
    else:
        # Higher in cleared group (0) or first week
        df_sorted["Direction"].loc[df_sorted["log2fold"]<0] = 'Decreased Risk'

        # Higher in recurred group (1) or second week
        df_sorted["Direction"].loc[df_sorted["log2fold"]>0] = 'Increased Risk'
    
    df_sorted = df_sorted.set_index("OTU")
    if plot:
        for metab in df_sorted.index.values:
            if df_sorted['padj'][metab] < .1:
                plot_metab_over_time(metab, df_sorted['padj'][metab], dl.keys[key], path_to_save + 'univariate_analysis/' + 
                                     key + '/' + str(week) + '_deseq2_')
    df_sorted.to_csv(path_to_save+'univariate_analysis/' + key + '/deseq2_' + key + str(week) + '.csv')
    plt.figure()
    plt.hist(df_sorted['padj'], bins = np.arange(0,1.05,.05))
    plt.xlabel('p-values')
    plt.ylabel('Frequency')
    plt.title(key + ', week ' + str(week))
    plt.savefig(path_to_save+'univariate_analysis/' + key + '/deseq2_' + key + str(week) + '.pdf')
    plt.close()
    

KeyboardInterrupt: 

## Univariate analysis for metabolites & SCFAs

In [6]:
# Univariate analysis for metabolites and SCFAs
path_to_save_data=save_path + '/GenFigure&AnalysesData/'

weeks = [0,1,2]
week_pairs = [(0,1),(1,2)]
plot = False
data_to_save = {}
for key in ['metabs', 'scfa']:
    data_to_save[key] = {}
    if not os.path.isdir(path_to_save + 'univariate_analysis/' + key):
        os.mkdir(path_to_save + 'univariate_analysis/' + key)
    for outcome in ['Recurrer', 'Non-recurrer']:
        if outcome not in data_to_save[key].keys():
            data_to_save[key][outcome] = {}
        for week_pair in week_pairs:
            x0, y0 = dl.week[key][week_pair[0]]['x'], dl.week[key][week_pair[0]]['y']
            x0 = x0.loc[y0.values == outcome]
            x1, y1 = dl.week[key][week_pair[1]]['x'], dl.week[key][week_pair[1]]['y']
            x1 = x1.loc[y1.values == outcome]

            xboth = list(set(x0.columns.values).intersection(set(x1.columns.values)))
            x = pd.concat([x0, x1])
            if 'scfa' in key:
                x = x.drop('Caproate', axis = 1)
            y = [week_pair[0]]*x0.shape[0] + [week_pair[1]]*x1.shape[0]
            y = pd.Series((np.array(y)==week_pair[1]).astype('float'), index = x.index.values)
            
            yy = pd.DataFrame(y, columns = ['Outcome'])
            both = pd.concat([x, yy], 1)
            data_to_save[key][outcome]['Week ' + str(week_pair[0]) + 'v' + str(week_pair[1])] = both
            
            df = univariate_w_chi2(x,y)
            df2 = univariate_w_chi2(x,y, method = 'ttest')
            if not os.path.isdir(path_to_save+'univariate_analysis/' + key ):
                os.mkdir(path_to_save+'univariate_analysis/' + key)

            df.insert(2,'Direction', np.zeros(df.shape[0]).astype(str))
            df2.insert(2,'Direction', np.zeros(df2.shape[0]).astype(str))

            # Higher in cleared group (0) or first week
            df["Direction"].loc[df["direction"].astype('float64')<0] = 'Higher in week ' + str(week_pair[0])
            df2["Direction"].loc[df2["direction"].astype('float64')<0] = 'Higher in week ' + str(week_pair[0])
            # Higher in recurred group (1) or second week
            df["Direction"].loc[df["direction"].astype('float64')>0] = 'Higher in week ' + str(week_pair[1])
            df2["Direction"].loc[df2["direction"].astype('float64')>0] = 'Higher in week ' + str(week_pair[1])

            df.to_csv(path_to_save+'univariate_analysis/' + key + '/ranksum_' + key + str(week_pair[0]) + '_' + 
                       str(week_pair[1]) + '_' + outcome + '.csv')
            df2.to_csv(path_to_save+'univariate_analysis/' + key + '/ttest_' + key + str(week_pair[0]) + '_' + 
                       str(week_pair[1]) + '_' + outcome + '.csv')
            plt.figure()
            plt.hist(df2['BH corrected'], bins = np.arange(0,1.05,.05))
            plt.xlabel('p-values')
            plt.ylabel('Frequency')
            plt.title(key + ', week ' + str(week_pair))
            plt.savefig(path_to_save+'univariate_analysis/' + key + '/ttest_' + key + str(week_pair[0]) + '_' + 
                       str(week_pair[1]) + '_' + outcome + '.pdf')
            plt.close()
#             with pd.ExcelWriter(path_save + 'Table 5 - Predictive Results.xlsx', mode = 'a') as writer:
#                 df.to_excel(writer, sheet_name = 'A. Results')
        
    for week in weeks:
        if week not in data_to_save[key].keys():
            data_to_save[key]['Week ' + str(week)] = {}

        if not isinstance(week, list):
            x, y = dl.week[key][week]['x'], dl.week[key][week]['y']
            
            yy = pd.Series((np.array(y)=='Recurrer').astype('float'), index = x.index.values)
            yy = pd.DataFrame(yy, columns = ['Outcome'])
            both = pd.concat([x, yy], 1)
            data_to_save[key]['Week ' + str(week)]['RvNR'] = both
            if 'scfa' in key:
                x = x.drop('Caproate', axis = 1)
            if 'toxin' in key:
                x = x.iloc[:,-4:]
            df = univariate_w_chi2(x,y)
            df2 = univariate_w_chi2(x,y, method = 'ttest')
        else:
            x,y,t = get_slope_data(dl.week[key], week)
            if 'scfa' in key:
                x = x.drop('Caproate', axis = 1)
            if 'toxin' in key:
                x = x.iloc[:,-4:]
            df = univariate_w_chi2(x,y)
            df2 = univariate_w_chi2(x,y,method = 'ttest')
        y = pd.DataFrame(y, columns = ['Outcome'])
        both = pd.concat([x, yy], 1)
        data_to_save[key]['Week ' + str(week)]['RvNR'] = both
            
        if not os.path.isdir(path_to_save+'univariate_analysis/' + key):
            os.mkdir(path_to_save+'univariate_analysis/' + key)
        if plot:
            for metab in df2.index.values:
                if df2['BH corrected'][metab] < .05:
                    try:
                        plot_metab_over_time(metab, df2['BH corrected'][metab], dl.keys[key], path_to_save + 'univariate_analysis/' + 
                                             key + '/' + str(week) + '_')
                        print('plotted ' + metab)
                    except:
                        plot_metab_over_time(metab, df2['BH corrected'][metab], dl.keys['metabs'], path_to_save + 'univariate_analysis/' + 
                                             key + '/' + str(week) + '_')
                        print('plotted ' + metab)                    
        
        df.insert(2,'Direction', np.zeros(df.shape[0]).astype(str))
        df2.insert(2,'Direction', np.zeros(df2.shape[0]).astype(str))
        
        # Higher in cleared group (0) or first week
        df["Direction"].loc[df["direction"].astype('float64')<0] = 'Decreased Risk'
        df2["Direction"].loc[df2["direction"].astype('float64')<0] = 'Decreased Risk'

        # Higher in recurred group (1) or second week
        df["Direction"].loc[df["direction"].astype('float64')>0] = 'Increased Risk'
        df2["Direction"].loc[df2["direction"].astype('float64')>0] = 'Increased Risk'
        
        df.to_csv(path_to_save+'univariate_analysis/' + key + '/ranksum_' + key + str(week) + '.csv')
        df2.to_csv(path_to_save+'univariate_analysis/' + key + '/ttest_' + key + str(week) + '.csv')
        plt.figure()
        plt.hist(df2['BH corrected'], bins = np.arange(0,1.05,.05))
        plt.xlabel('p-values')
        plt.ylabel('Frequency')
        plt.title(key + ', week ' + str(week))
        plt.savefig(path_to_save+'univariate_analysis/' + key + '/ttest_' + key + str(week) + '.pdf')
        plt.close()
        
        
for key in data_to_save.keys():
    for key2 in data_to_save[key].keys():
        for key3 in data_to_save[key][key2].keys():
            val = pd.DataFrame(data_to_save[key][key2][key3])
            if os.path.isfile(path_to_save_data + '/' + 'univ_analysis_data.xlsx'):
                with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_data.xlsx', mode='a', engine='openpyxl') as writer:
                    val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))
            else:
                with pd.ExcelWriter(path_to_save_data + '/' + 'univ_analysis_data.xlsx', mode = 'w', engine='openpyxl') as writer:
                    val.to_excel(writer, sheet_name='_'.join([key, key2, key3.replace(' ','_')]))

## Save compact univariate analysis to supplemental tables

In [27]:
from string import ascii_uppercase
path_univariate = path_to_save + 'univariate_analysis/'
path_save = save_path + '/Supplemental Tables/'
taxa_labels = pd.read_csv(path_to_save + 'scripts/inputs/' + 'taxa_labels.csv', index_col = [0])

k = 3
for dtype in os.listdir(path_univariate):
    if dtype == 'toxin':
        continue
    labs = []
    df_out = {}

    fname = 'Table 2. Univariate Analysis.xlsx'

    i = 0
    if dtype == '.DS_Store':
        continue
        
    if '.csv' in dtype:
        continue
    for file in os.listdir(path_univariate + dtype):
        if 'kruskal' in file or 'ranksum' in file or '[' in file or '.DS_Store' in file:
            continue
        if '.csv' not in file:
            continue
#         print(i)
#         print(file)
        timepoint = file.split('.')[0].split(dtype)[1]
        if 'rer' in timepoint:
            if dtype == '16s':
                label = timepoint.split('rer')[0] + 'rer;Week' + 'vs'.join(timepoint.split('rer')[1].split('_'))
            else:
                label = timepoint.split('_')[-1] + ';Week' + 'vs'.join(timepoint.split('_')[:-1])
            lab_out = ('Intra-group',label)
        else:
            label = 'Week' + timepoint + ';R_vs_NR'
            lab_out = ('Inter-group',label)
        
        df = pd.read_csv(path_univariate + dtype + '/' + file, index_col = [0])
        labs.append(lab_out)
        if dtype == '16s':
            df = df.sort_values(by = 'padj')
            if np.sum(df['padj']<0.1)==0:
                df = df.iloc[:10,:]
            else:
                df = df.loc[df['padj']<0.1]
            seq_labels = taxa_labels['labels'].loc[df.index.values]
            for ix in df.index.values:
                df_out[(lab_out[0], lab_out[1], ix)] = {'Labels': seq_labels.loc[ix], 'FDR': df['padj'].loc[ix], 
                                                        'Direction': df['Direction'].loc[ix], 
                          'log2fold': df['log2fold'].loc[ix],
                         'Taxonomy-RDP': taxa_labels['taxa_rdp'].loc[ix], 
                          }
#             df_out[lab_out] = pd.DataFrame(df_out[lab_out])
#             df_out[lab_out] = df_out[lab_out].set_index('Labels')
        elif dtype == 'metabs':
            df = df.sort_values(by = 'BH corrected')
            if np.sum(df['BH corrected']<0.1)==0:
                df = df.iloc[:10,:]
            else:
                df = df.loc[df['BH corrected']<0.1]
            for ix in df.index.values:
                df_out[(lab_out[0], lab_out[1], ix)] = {'FDR': df['BH corrected'].loc[ix], 
                                                        'Direction': df['Direction'].loc[ix], 
                          'test statistic': df['test statistic'].loc[ix], 
                         'Super Pathway': dl.col_mat_mets['SUPER_PATHWAY'].loc[ix],
                         'Sub Pathway': dl.col_mat_mets['SUB_PATHWAY'].loc[ix]}
#             df_out[lab_out] = pd.DataFrame(df_out[lab_out])
#             df_out[lab_out] = df_out[lab_out].set_index('names')
        else:
            df = df.sort_values(by = 'BH corrected')
#             df = df.loc[df['BH corrected']<0.1]
            for ix in df.index.values:
                df_out[(lab_out[0], lab_out[1], ix)] = {'FDR': df['BH corrected'].loc[ix], 
                                   'Direction': df['Direction'].loc[ix], 
                               'test statistic': df['test statistic'].loc[ix]}
            
        
    df_out = pd.DataFrame(df_out).T.sort_index()
    if os.path.exists(path_save + fname):
        with pd.ExcelWriter(path_save + fname, mode = 'a') as writer:
            df_out.to_excel(writer, sheet_name = dtype.capitalize())
    else:
        with pd.ExcelWriter(path_save + fname) as writer:
            df_out.to_excel(writer, sheet_name = dtype.capitalize())
#         i+= 1
    k += 1

## Save full univariate analysis to Data folder

In [40]:
from string import ascii_uppercase
path_univariate = path_to_save + 'univariate_analysis/'
path_save = save_path + '/Data/Univariate analyses/'
if not os.path.isdir(path_save):
    os.mkdir(path_save)
taxa_labels = pd.read_csv(path_to_save + 'scripts/inputs/' + 'taxa_labels.csv', index_col = [0])

k = 3
for dtype in os.listdir(path_univariate):
    fname = 'Table ' + str(k + 1) + '-' + dtype.capitalize() + ' Univariate Analysis.xlsx'

    
    if dtype == '.DS_Store':
        continue
    if '.csv' in dtype:
        os.rename(path_univariate + dtype, path_save + dtype)
        continue
    i = 0
    for file in os.listdir(path_univariate + dtype):
        if 'kruskal' in file or 'ranksum' in file or '[' in file or '.DS_Store' in file:
            continue
        if '.csv' not in file:
            continue
#         print(i)
#         print(file)
        
        df = pd.read_csv(path_univariate + dtype + '/' + file, index_col = [0])
        if dtype == '16s':
            seq_labels = taxa_labels['labels'].loc[df.index.values]
            df_out = {'FDR': df['padj'], 'Direction': df['Direction'], 
                      'log2fold': df['log2fold'],'Taxonomy-Silva': taxa_labels['taxa_silva'].loc[df.index.values], 
                     'Taxonomy-RDP': taxa_labels['taxa_rdp'].loc[df.index.values], 
                      'Sequences': df.index.values, 'Labels': seq_labels}
            df_out = pd.DataFrame(df_out)
            df_out = df_out.set_index('Labels')
        elif dtype == 'metabs':
            df_out = {'FDR': df['BH corrected'], 'Direction': df['Direction'], 
                      'test statistic': df['test statistic'], 'PubChem#': dl.col_mat_mets['PUBCHEM'].loc[df.index.values],
                     'KEGG#': dl.col_mat_mets['PUBCHEM'].loc[df.index.values], 'HMDB#': 
                      dl.col_mat_mets['HMDB'].loc[df.index.values], 
                     'Super Pathway': dl.col_mat_mets['SUPER_PATHWAY'].loc[df.index.values],
                     'Sub Pathway': dl.col_mat_mets['SUB_PATHWAY'].loc[df.index.values], 'names':df.index.values}
            df_out = pd.DataFrame(df_out)
            df_out = df_out.set_index('names')
        elif dtype == 'toxin':
            df_out = {'FDR': df['BH corrected'], 'Direction': df['Direction'], 'test statistic': df['test statistic'],
                     'method': df['method']}
            df_out = pd.DataFrame(df_out)
        else:
            df_out = {'FDR': df['BH corrected'], 'Direction': df['Direction'], 'test statistic': df['test statistic']}
            df_out = pd.DataFrame(df_out)
        
        timepoint = file.split('.')[0].split(dtype)[1]

        if 'rer' in timepoint:
            if dtype == '16s':
                label = timepoint.split('rer')[0] + 'rer;Week' + 'vs'.join(timepoint.split('rer')[1].split('_'))
            else:
                label = timepoint.split('_')[-1] + ';Week' + 'vs'.join(timepoint.split('_')[:-1])
        else:
            label = 'Week' + timepoint + ';R_vs_NR'
        
        nm = ascii_uppercase[i]
        
        if os.path.exists(path_save + fname):
            with pd.ExcelWriter(path_save + fname, mode = 'a') as writer:
                df_out.to_excel(writer, sheet_name = nm + '.' + label)
        else:
            with pd.ExcelWriter(path_save + fname) as writer:
                df_out.to_excel(writer, sheet_name = nm + '.' + label)
        i+= 1
    k += 1